# Interpreting Linear Regression

In this notebook we will continue with our car price prediction example and explore the methods we use to interpret and evaluate the results of our linear regression model. We will begin by reproducing the results from the previous notebook.

As usual, we first import the required packages and dataset.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plot
import statsmodels.api as stats
import numpy as np

In [ ]:
carprice_df = pd.read_csv('CarPrice_Assignment.csv')

We then remove the independent variables with too many unique categories or have high correlations with other independent variables.

In [ ]:
carprice_df = carprice_df.drop(columns=['car_ID', 'CarName', 'carlength', 'carwidth', 'highwaympg'])

Next we one-hot-encode the categorical variables so each category in a categorical variable becomes its own binary column - this converts categorical variables to numeric. We also remove more highly correlated variables after including the one-hot-encoded variables.

In [ ]:
dummy = pd.get_dummies(carprice_df.select_dtypes(include='object'), drop_first=True)

In [ ]:
carprice_df = pd.concat([carprice_df.select_dtypes(exclude='object'), dummy], axis=1)

In [ ]:
carprice_df = carprice_df.drop(columns=['compressionratio', 'drivewheel_fwd', 'enginetype_rotor', 'fuelsystem_4bbl', 'fuelsystem_idi'])

We then split our data into train and test sets, and add our constant column.

In [ ]:
train_df=carprice_df.sample(frac=0.7, random_state=101) 
test_df=carprice_df.drop(train_df.index)

In [ ]:
Y_train = train_df.price
X_train = stats.add_constant(train_df.drop(columns=['price']))

We then fit our model to the training data

In [ ]:
model_carprice = stats.OLS(Y_train, X_train)
results_carprice = model_carprice.fit()

We finally produce our test set predictions 

In [ ]:
Y_test = test_df.price
test_df = stats.add_constant(test_df)
X_test = test_df[X_train.columns]

In [ ]:
test_predictions = results_carprice.predict(X_test)

# p-Values and Coefficients

### p-Values

The statsmodels summary output we produced previously provides us with the p-values associated with each of our regression coefficients. These are given in the P>|t| column. Scikit-Learn does not offer any functionality to calcualte p-values so statsmodels should be used if you want to test the significance of coefficients.

In [ ]:
print(results_carprice.summary())

We can see that many of the independent variables are not significant at the 5% level i.e. they have p-value >0.05. We can discard these variables from the model and re-train our model with only the statistically significant variables. We can repeat this until all our independent variables are significant.

### Re-training the Model

In [ ]:
Y_train_new = train_df.price
X_train_new = stats.add_constant(train_df[['enginesize', 
                                    'stroke', 
                                    'peakrpm', 
                                    'fueltype_gas', 
                                    'carbody_hardtop', 
                                    'carbody_hatchback', 
                                    'enginelocation_rear',
                                    'enginetype_ohc',
                                    'cylindernumber_five',
                                    'cylindernumber_four',
                                    'cylindernumber_six']])

In [ ]:
model_carprice_new = stats.OLS(Y_train_new, X_train_new)
results_carprice_new = model_carprice_new.fit()

In [ ]:
print(results_carprice_new.summary())

All of our independent variables now have p-values below 0.05 (except const which we are not concerned whether this is zero or not) and we have reduced the complexity of our model by reducing the number of variables.

In [ ]:
Y_test_new = test_df.price
X_test_new = test_df[X_train_new.columns]

In [ ]:
test_predictions_new = results_carprice_new.predict(X_test_new)

In [ ]:
plot.scatter(test_predictions_new, Y_test_new)
plot.plot([5000, 50000], [5000, 50000], c='k', ls='--')
plot.xlabel('Predicted Price [$]')
plot.ylabel('Observed Price [$]')
plot.show()

### Coefficients

The coefficients tell us how much the target variable changes for a one unit change in the corresponding independent variable when all the other independent variables are held fixed. We can get a measure of which independent variable contributes the most change to the target variable if we first scale all the independent variables to remove the units. This is known as standardization as we discussed in the theory lesson.

In [ ]:
X_train_scale = ((X_train_new - X_train_new.mean())/X_train_new.std()).drop(columns=['const'])

In [ ]:
X_train_scale.std()

In [ ]:
X_train_scale.mean().round(2)

As we can see the independent variables now all have mean zero and a standard deviation of one and will will therefore be able to compare coefficient values. We will now refit the model. Scikit-Learn also contains a function [StandardScaler()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) which will perform this standardization.

In [ ]:
X_train_scale = stats.add_constant(X_train_scale)

We add our constant back in as we removed this in the above step since standardization doesn't work for a constant value. This is because both the numerator and denominator are zero and dividing zero by zero doesn't make sense to us. We then fit our model to this standardized data. 

In [ ]:
model_carprice_scale = stats.OLS(Y_train_new, X_train_scale)
results_carprice_scale = model_carprice_scale.fit()

In [ ]:
print(results_carprice_scale.summary())

The magnitudes of the coefficients now tell us how much they contribute to the change in price relative to each other. We can plot these in a bar chart using the [df.plot.bar()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.bar.html) method. We use the .abs() method to make all the values positive since we are only concerned with the magnitudes.

In [ ]:
results_carprice_scale.params.drop(index=['const']).abs().sort_values(ascending=False).plot.bar()

Here we can see that a change in engine size contributes the biggest change to the price while whether the car is a hatchback or not contributes the least. In a sense this gives us how important each feature is relatively to the model.

# Residuals and Residual Plots

We can assess our model using the residuals between the predicted values and the observed values. statsmodels results instances have a method [.resid](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.RegressionResults.html) for calculating these residuals. If we are using Scikit-Learn then we can manually calculate the residuals by subtracting the predicted values from the observed values.

In [ ]:
results_carprice_new.resid

We can then plot these residuals against our predicted values for the training data and observe if we see any non-random pattern in the data. 

In [ ]:
plot.scatter(results_carprice_new.fittedvalues, results_carprice_new.resid)
plot.plot([5000,50000], [0,0], c='k', ls='--')

As we can see in the plot above, there seems to be no discernible pattern in the residuals which is promising.
#### Durbin-Watson Test
The results summary above also provides us with a [Durbin-Watson](https://www.investopedia.com/terms/d/durbin-watson-statistic.asp) metric. This tests the residuals for any possible autocorrelation and will have a value between 0 and 4. A value of 2 means no autocorrelation and values between 1.5 and 2.5 are relatively normal when the data is not especially autocorrelated. The summary gives us a value of 1.85 which is not a cause for concern. 

#### Breusch-Pagan test
We can also do a quick test for heteroskedasticity using the [Breusch-Pagan](https://www.statology.org/breusch-pagan-test/) test. Here we calculate the value $nR^{2}$ where $n$ is the number of datapoints in the model (in our case 144) and $R^{2}$ is the coefficient of determination. This is given to us in the results summary where it is shown as R-squared (our value is 0.883). This value is distributed as a $\chi^{2}$ variable with degrees of freedom of $n-p-1$ where $p$ is our number of independent variables in the model (the -1 is for the intercept value). From this we can get a p-value where our null hypothesis is that the data is homoscedastic (constant variance) i.e. the probabiltity that our data is homoscedastic. If our p-value is less that 0.05 then it is significant and our data is likely heteroskedastic.
From our data we get a value for $nR^{2}$ of 127 and we have $144-11-1=132$ degrees of freedom. We can use these two values to [calculate](https://www.statology.org/chi-square-p-value-calculator/) our p-value. For our data we get a p-value of 0.6 and as such this is not significant and we can treat the data as homoscedastic. 

As we are now satisfied that a linear regression model is appropriate and our data doesn't violate any assumptions required to use ordinary least squares, we can move on to evaluating how good our model is with various metrics. 

# Evaluating Linear Regression

statsmodels allows us to easily calculate the metrics we have discussed ([mean square error](https://www.statsmodels.org/stable/generated/statsmodels.tools.eval_measures.mse.html), [root mean square error](https://www.statsmodels.org/stable/generated/statsmodels.tools.eval_measures.rmse.html), [mean absolute error](https://www.statsmodels.org/stable/generated/statsmodels.tools.eval_measures.meanabs.html), [$R^{2}$](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.RegressionResults.rsquared.html) and [adjusted $R^{2}$](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLSResults.rsquared_adj.html)) We can calculate these for either/both the training data and the test data.

In [ ]:
train_mse = stats.tools.eval_measures.mse(Y_train_new, results_carprice_new.fittedvalues)
print('The training dataset mean square error is {}'.format(train_mse.round(1)))

In [ ]:
train_rmse = stats.tools.eval_measures.rmse(Y_train_new, results_carprice_new.fittedvalues)
print('The training dataset root mean square error is {}'.format(train_rmse.round(1)))

In [ ]:
train_mae = stats.tools.eval_measures.meanabs(Y_train_new, results_carprice_new.fittedvalues)
print('The training dataset mean absolute error is {}'.format(train_mae.round(1)))

In [ ]:
train_r2 = results_carprice_new.rsquared
print('The training dataset coefficient of determination is {}'.format(train_r2.round(3)))

In [ ]:
train_r2_adj = results_carprice_new.rsquared_adj
print('The training dataset adjusted coefficient of determination is {}'.format(train_r2_adj.round(3)))

We can see above in the root mean square error that our predictions are typically around \\$3000 off from the observed values and that the data explains 88.3% of the variation from the coefficient of determination. 

The coefficient of determination and root mean square error for our simple linear regression model from Chapter 2 were 0.75 and \\$4000 respectively so we are explaining more of the variation and getting better (on average) predictions with our multiple independent variables. As the adjusted coefficient of determination is similar here we can be reasonably confident that the addition of more independent variables is not simply adding random fluctuations to the model.

### Scikit-Learn

Again we can repeat using the Scikit-Learn package and calculate our metrics. We will use the same independent variables we used above after calculating the p-values.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
Y = carprice_df.price
X = carprice_df[['enginesize', 
                 'stroke', 
                 'peakrpm', 
                 'fueltype_gas', 
                 'carbody_hardtop', 
                 'carbody_hatchback', 
                 'enginelocation_rear',
                 'enginetype_ohc',
                 'cylindernumber_five',
                 'cylindernumber_four',
                 'cylindernumber_six']]

As before, we split our data into test and train sets and fit our model on the training data.

In [ ]:
sk_X_train, sk_X_test, sk_Y_train, sk_Y_test = train_test_split(X, Y, test_size=0.3, random_state=97)

In [ ]:
regressor = LinearRegression()  
regressor.fit(sk_X_train, sk_Y_train)

In [ ]:
sk_intercept_carprice = regressor.intercept_
sk_engsize_coeffs = regressor.coef_
sk_ssr_carprice = np.sum((sk_Y_train-regressor.predict(sk_X_train))**2)

Here we use our model to make our predictions for both the training and test data. 

In [ ]:
sk_train_predictions = regressor.predict(sk_X_train)
sk_test_predictions = regressor.predict(sk_X_test)

We can extract the mean squared error, mean absolute error and $R^{2}$ using Scikit-Learn function from the module sklearn.metrics. Adjusted $R^{2}$ needs to be calculated manually from $R^{2}$ and the root mean square error can be taken as the square root of the MSE. These functions take our observed target values and predicted values as arguments.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
sk_train_mse = mean_squared_error(sk_Y_train, sk_train_predictions)
print('The training dataset mean square error is {}'.format(sk_train_mse.round(1)))

In [ ]:
sk_train_rmse = np.sqrt(sk_train_mse)
print('The training dataset root mean square error is {}'.format(sk_train_rmse.round(1)))

In [ ]:
sk_train_mae = mean_absolute_error(sk_Y_train, sk_train_predictions)
print('The training dataset mean absolute error is {}'.format(sk_train_mae.round(1)))

In [ ]:
sk_train_r2 = r2_score(sk_Y_train, sk_train_predictions)
print('The training dataset coefficient of determination is {}'.format(sk_train_r2.round(3)))

we calculate the adjusted $R^{2}$ using the following formula (where we have substituted the equation for $R^{2}$ into our equation for the adjusted $R^{2}$ shown in the theory lesson).

$ 1-\frac{(1-R^2)(n-1)}{(n-p-1)}$

- n is the number of observations in our data
- p is the number of independent variables we have

In [ ]:
n = sk_X_train.shape[0]
p = sk_X_train.shape[1]
sk_train_r2_adj = 1-(1-sk_train_r2)*(n-1)/(n-p-1)
print('The training dataset adjusted coefficient of determination is {}'.format(sk_train_r2_adj.round(3)))

### Test Data

Scikit-learn also easily allows us to evaluate these metrics on our test data to see how well our model performs on unseen data. This is key when we want to use our model to predict new values. Below we calculate our $R^{2}$ value from the test data.

In [ ]:
sk_test_r2 = r2_score(sk_Y_test, sk_test_predictions)
print('The test dataset coefficient of determination is {}'.format(sk_test_r2.round(3)))

As we can see, our model works well on the test data, explaining 84% of the variance. It is typical that even in a good model the score on the test data will be slightly lower than on the training data as the model was built on the training data. If the test score is significantly lower than the train score then this is an indicator that our model is overfitting the training data as the model is too specific to the data it is trained on.